<a href="https://colab.research.google.com/github/shreyas-rc/GEE_Codes_V.1.2/blob/main/Parallel_Processing_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Below is a Javascript coder editor code of Google Earth Engine that helps download data parallelly  

In [ ]:
"""
Code Starts Here ---->

//The Lines 3 to 17 are imports such that, 4-16 represents a geometry I drew, which covers the entire India as a rectangle,
//It is necessary to draw this rectangle based geometry because, if we use India as a Bounding box, since it has n-number of
//Vertices it gives rise to server timeout. Therefore one simple rectangle can solve the problem.
//I've also downloaded shapefile for India and uploaded into assests and used it here, feel free to use any shapefile that you want.

/**** Start of imports. If edited, may not auto-convert in the playground. ****/
var one =
    /* color: #00ffff */
    /* shown: false */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[63.62235869665927, 38.7274640420804],
          [63.62235869665927, 3.830203655296874],
          [99.83329619665926, 3.830203655296874],
          [99.83329619665926, 38.7274640420804]]], null, false),
    India = ee.FeatureCollection("projects/.../assets/Shapefiles/India_Boundary");

/***** End of imports. If edited, may not auto-convert in the playground. *****/

// Select the MODIS LST image collection - here I'm using a 8 - day average
var lst = ee.ImageCollection('MODIS/006/MOD11A2')
  .select(['LST_Day_1km', 'LST_Night_1km']) //Selecting 2 bands Day and Night LST (Check the bands present in Image collection description)
  .filterDate('2000-01-01', '2024-12-31') //Dates
  .filterBounds(India); //Filtering data that touches or is within India boundary, actual clipping will take place at the bottom

lst = lst.filter(ee.Filter.calendarRange(12,12,'month')) //I'm filtering the data for the months of december (you can choose the months)

// Calculate the diurnal temperature range (DTR) for each image
var dtr = lst.map(function(image) {
  var day = image.select('LST_Day_1km').multiply(0.02).subtract(273.15); //Converting LST from Kelvin to Degree Celsius
  var night = image.select('LST_Night_1km').multiply(0.02).subtract(273.15); //Converting LST from Kelvin to Degree Celsius
  var dtr = day.subtract(night).clip(India) //Clipping to India
  return image.addBands(dtr.rename('DTR'));
}).select('DTR');
var meanly = dtr.mean() //Necessary to generate mean of each december


// Define export parameters
//Change the parameters according to your needs
var exportParams = {
  driveFolder: 'MODIS_DTR',
  region: one, //This is the rectangle we choose, easy to export shapefiles in a rectangle
  scale: 1000, //Pixel size of MODIS
  maxPixels: 1e13
};

// Loop over each year and export the DTR image
//I'm using a for loop to loop over each year and get december month images
var years = ee.List.sequence(2000, 2024); //years 2000 to 2024
years.getInfo().forEach(function(year) {
  var startDate = ee.Date.fromYMD(year, 1, 1);
  var endDate = startDate.advance(1, 'year');
  var image = dtr.filterDate(startDate, endDate).mean().clip(India);
  image = image.unmask(meanly)
  var exportName = 'MODIS_DTR_Dec' + year; //If you are using other data other than MODIS, change name accordingly.
  Export.image.toDrive({
    image: image,
    description: exportName,
    fileNamePrefix: exportName,
    folder: exportParams.driveFolder,
    region: exportParams.region,
    crs: 'EPSG:4326',
    scale: exportParams.scale,
    maxPixels: exportParams.maxPixels
  });
});

<---Code ends
"""

### Refer Docs here for more information on how to execute it efficiently
https://github.com/shreyas-rc/GEE_Codes_V.1.2/blob/main/Parallel_Processing_Doc.pdf